In [0]:
files= dbutils.fs.ls('/Volumes/workspace/bookstore_eng_pro/dataset/kafka-raw/')
display(files)

In [0]:
df_raw = spark.read.json('/Volumes/workspace/bookstore_eng_pro/dataset/kafka-raw/')
display(df_raw)

In [0]:
from pyspark.sql import functions as F

def process_bronze():
     schema = "key BINARY, value BINARY, topic STRING, partition LONG, offset LONG, timestamp LONG"

     query = (spark.readStream
                        .format("cloudFiles")
                        .option("cloudFiles.format","json")
                        .schema(schema)
                        .load("/Volumes/workspace/bookstore_eng_pro/dataset/kafka-raw/")
                        .withColumn("timestamp",(F.col("timestamp")/1000).cast("timestamp"))
                        .withColumn("year_month",F.date_format("timestamp","yyyy-MM"))
                    .writeStream
                        .option("checkpointLocation","/Volumes/workspace/bookstore_eng_pro/checkpoints/bronze")
                        .option("mergeSchema",True)
                        .trigger(availableNow=True)
                        .table("raw_bronze_kakfa"))
     
     query.awaitTermination()
                
 


In [0]:
process_bronze()

In [0]:
batch_df =spark.table("workspace.default.raw_bronze_kakfa")

display(batch_df)

In [0]:
%sql
select * from raw_bronze_kakfa